In [1]:
import os
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer as Imputer
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import gc

In [3]:


FILE_PATH = os.path.join("datasets")

def load_csv_data(csvName, path=FILE_PATH, parse_dates=False):
    csv_path = os.path.join(path, csvName)
    if parse_dates:
        return pd.read_csv(csv_path, parse_dates=['timestamp'])
    else:
        return pd.read_csv(csv_path)
        


# load csv
building_metadata = load_csv_data("building_metadata.csv")

#train = load_csv_data("train.csv", parse_dates=True)
#test = load_csv_data("test.csv", parse_dates=True)

#weather_train = load_csv_data("weather_train.csv", parse_dates=True)
#weather_test = load_csv_data("weather_test.csv", parse_dates=True)

gc.collect()

20

In [2]:
building_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
site_id        1449 non-null int64
building_id    1449 non-null int64
primary_use    1449 non-null object
square_feet    1449 non-null int64
year_built     675 non-null float64
floor_count    355 non-null float64
dtypes: float64(2), int64(3), object(1)
memory usage: 68.0+ KB


In [3]:
building_metadata.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [4]:
building_metadata["primary_use"].unique()

array(['Education', 'Lodging/residential', 'Office',
       'Entertainment/public assembly', 'Other', 'Retail', 'Parking',
       'Public services', 'Warehouse/storage', 'Food sales and service',
       'Religious worship', 'Healthcare', 'Utility', 'Technology/science',
       'Manufacturing/industrial', 'Services'], dtype=object)

In [5]:
train.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01,0.0
1,1,0,2016-01-01,0.0
2,2,0,2016-01-01,0.0
3,3,0,2016-01-01,0.0
4,4,0,2016-01-01,0.0


In [6]:
train["meter"].unique()

array([0, 3, 1, 2])

In [7]:
train["meter_reading"].unique()

array([   0.    ,   53.2397,   43.0013, ..., 3006.82  ,  852.77  ,
         55.6241])

In [8]:
tmp = pd.Series(train["meter_reading"], dtype=np.float32)


In [9]:
train.isnull().sum()

building_id      0
meter            0
timestamp        0
meter_reading    0
dtype: int64

In [10]:
building_metadata.isnull().sum()

site_id           0
building_id       0
primary_use       0
square_feet       0
year_built      774
floor_count    1094
dtype: int64

In [11]:
train[["building_id", "meter_reading"]]

,building_id,meter_reading
0,0,0.000
1,1,0.000
2,2,0.000
3,3,0.000
4,4,0.000
...,...,...
20216095,1444,8.750
20216096,1445,4.825
20216097,1446,0.000
20216098,1447,159.575


In [83]:
# TRAIN
# tmp = pd.merge(train[["timestamp", "building_id", "meter", "meter_reading"]], building_metadata[["building_id", "primary_use"]],
#         how="left", on = "building_id")

## test
tmp = pd.merge(data[["timestamp", "building_id", "meter"]], building_metadata[["building_id", "primary_use"]],
        how="left", on = "building_id")

In [84]:
tmp.head()

,timestamp,building_id,meter,primary_use
0,2017-01-01,0,0,Education
1,2017-01-01,1,0,Education
2,2017-01-01,2,0,Education
3,2017-01-01,3,0,Education
4,2017-01-01,4,0,Education


In [85]:
tmp_pr = tmp.copy()
i:int = 1
for pr_val in tmp["primary_use"].unique():
    print(pr_val)
    tmp_pr["primary_use"][tmp_pr["primary_use"] == pr_val] = i
    i += 1

Education


/Users/takahisahirota/Documents/repo/ml/env/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Lodging/residential
Office
Entertainment/public assembly
Other
Retail
Parking
Public services
Warehouse/storage
Food sales and service
Religious worship
Healthcare
Utility
Technology/science
Manufacturing/industrial
Services


In [86]:
tmp_pr.head()

,timestamp,building_id,meter,primary_use
0,2017-01-01,0,0,1
1,2017-01-01,1,0,1
2,2017-01-01,2,0,1
3,2017-01-01,3,0,1
4,2017-01-01,4,0,1


In [87]:
tmp = tmp_pr.assign(primary_use_int = tmp_pr['primary_use'].astype(np.int64))

In [88]:
tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 5 columns):
timestamp          datetime64[ns]
building_id        int64
meter              int64
primary_use        object
primary_use_int    int64
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 1.9+ GB


In [89]:
primary_use_int = tmp.copy()

In [90]:
primary_use_int.head()

,timestamp,building_id,meter,primary_use,primary_use_int
0,2017-01-01,0,0,1,1
1,2017-01-01,1,0,1,1
2,2017-01-01,2,0,1,1
3,2017-01-01,3,0,1,1
4,2017-01-01,4,0,1,1


In [19]:
# join_columns = []
# for tr_column in list(train):
#     if tr_column == "building_id":
#         continue
    
    
#     for bu_column in list(building_metadata):
#         if bu_column == "building_id":
#             continue
#         tmp = pd.merge(train[["building_id", tr_column]], building_metadata[["building_id", bu_column]],
#         how="left", on = "building_id")
#         name = tr_column + bu_column
#         print(name)
#         join_columns.append([name, tmp])
#         #tmp.to_pickle('tmpdata/' + [tr_column + bu_column] + '.zip')



In [20]:
#join_columns

NameError: name 'join_columns' is not defined

In [ ]:
# for join in join_columns:
#     print(join[0])
#     print(join[1].corr())

In [91]:
## train and building important datasets
# train_building = pd.merge(train[["building_id", "meter_reading"]], building_metadata[["building_id", "year_built", "floor_count"]],
#                              how="left", on = "building_id")

## test
train_building = pd.merge(train[["building_id"]], building_metadata[["building_id", "year_built", "floor_count"]],
                             how="left", on = "building_id")

In [92]:
weather_train.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [93]:
train.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01,0.0
1,1,0,2016-01-01,0.0
2,2,0,2016-01-01,0.0
3,3,0,2016-01-01,0.0
4,4,0,2016-01-01,0.0


In [24]:
building_metadata.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [25]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
building_id      int64
meter            int64
timestamp        datetime64[ns]
meter_reading    float64
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 616.9 MB


In [26]:
weather_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
site_id               139773 non-null int64
timestamp             139773 non-null datetime64[ns]
air_temperature       139718 non-null float64
cloud_coverage        70600 non-null float64
dew_temperature       139660 non-null float64
precip_depth_1_hr     89484 non-null float64
sea_level_pressure    129155 non-null float64
wind_direction        133505 non-null float64
wind_speed            139469 non-null float64
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 9.6 MB


In [27]:
building_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
site_id        1449 non-null int64
building_id    1449 non-null int64
primary_use    1449 non-null object
square_feet    1449 non-null int64
year_built     675 non-null float64
floor_count    355 non-null float64
dtypes: float64(2), int64(3), object(1)
memory usage: 68.0+ KB


In [94]:
## train
# train_with_site_id = pd.merge(train, building_metadata[["site_id", "building_id"]], how="left", on = "building_id")

## test
train_with_site_id = pd.merge(test, building_metadata[["site_id", "building_id"]], how="left", on = "building_id")

In [95]:
train_with_site_id.head()

,row_id,building_id,meter,timestamp,site_id
0,0,0,0,2017-01-01,0
1,1,1,0,2017-01-01,0
2,2,2,0,2017-01-01,0
3,3,3,0,2017-01-01,0
4,4,4,0,2017-01-01,0


In [96]:
train_with_site_id.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 5 columns):
row_id         int64
building_id    int64
meter          int64
timestamp      datetime64[ns]
site_id        int64
dtypes: datetime64[ns](1), int64(4)
memory usage: 1.9 GB


In [97]:
weather_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
site_id               139773 non-null int64
timestamp             139773 non-null datetime64[ns]
air_temperature       139718 non-null float64
cloud_coverage        70600 non-null float64
dew_temperature       139660 non-null float64
precip_depth_1_hr     89484 non-null float64
sea_level_pressure    129155 non-null float64
wind_direction        133505 non-null float64
wind_speed            139469 non-null float64
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 9.6 MB


In [98]:
weather_train.isnull().sum()

site_id                   0
timestamp                 0
air_temperature          55
cloud_coverage        69173
dew_temperature         113
precip_depth_1_hr     50289
sea_level_pressure    10618
wind_direction         6268
wind_speed              304
dtype: int64

In [102]:
# ttrain
#weather_train_nonull = weather_train.drop(["cloud_coverage", "precip_depth_1_hr"], axis=1)

# test
weather_train_nonull = weather_test.drop(["cloud_coverage", "precip_depth_1_hr"], axis=1)

In [103]:
weather_train_nonull.head()

,site_id,timestamp,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed
0,0,2017-01-01 00:00:00,17.8,11.7,1021.4,100.0,3.6
1,0,2017-01-01 01:00:00,17.8,12.8,1022.0,130.0,3.1
2,0,2017-01-01 02:00:00,16.1,12.8,1021.9,140.0,3.1
3,0,2017-01-01 03:00:00,17.2,13.3,1022.2,140.0,3.1
4,0,2017-01-01 04:00:00,16.7,13.3,1022.3,130.0,2.6


In [104]:
def month(x):
    return x.month

def day(x):
    return x.day

def hour(x):
    return x.hour

month = weather_train_nonull["timestamp"].map(month)
day = weather_train_nonull["timestamp"].map(day)
hour = weather_train_nonull["timestamp"].map(hour)

weather_train_nonull["month"] = month
weather_train_nonull["day"] = day
weather_train_nonull["hour"] = hour

# from sklearn.impute import SimpleImputer
# si = SimpleImputer(missing_values=np.nan, strategy='mean')
# weather_train_nonull = si.fit_transform(weather_train_nonull)

In [105]:
weather_train_nonull.head()

,site_id,timestamp,air_temperature,dew_temperature,sea_level_pressure,wind_direction,wind_speed,month,day,hour
0,0,2017-01-01 00:00:00,17.8,11.7,1021.4,100.0,3.6,1,1,0
1,0,2017-01-01 01:00:00,17.8,12.8,1022.0,130.0,3.1,1,1,1
2,0,2017-01-01 02:00:00,16.1,12.8,1021.9,140.0,3.1,1,1,2
3,0,2017-01-01 03:00:00,17.2,13.3,1022.2,140.0,3.1,1,1,3
4,0,2017-01-01 04:00:00,16.7,13.3,1022.3,130.0,2.6,1,1,4


In [106]:
train_with_site_id.head()

,row_id,building_id,meter,timestamp,site_id
0,0,0,0,2017-01-01,0
1,1,1,0,2017-01-01,0
2,2,2,0,2017-01-01,0
3,3,3,0,2017-01-01,0
4,4,4,0,2017-01-01,0


In [107]:
train_pre1 = pd.merge(train_with_site_id, weather_train_nonull, how="left", on=["site_id", "timestamp"])
train_pre2 = train_pre1.drop(["air_temperature", "dew_temperature", "sea_level_pressure", "wind_direction"], axis=1)
gc.collect()


60

In [108]:
train_pre2.head()

,row_id,building_id,meter,timestamp,site_id,wind_speed,month,day,hour
0,0,0,0,2017-01-01,0,3.6,1.0,1.0,0.0
1,1,1,0,2017-01-01,0,3.6,1.0,1.0,0.0
2,2,2,0,2017-01-01,0,3.6,1.0,1.0,0.0
3,3,3,0,2017-01-01,0,3.6,1.0,1.0,0.0
4,4,4,0,2017-01-01,0,3.6,1.0,1.0,0.0


In [109]:
train_pre2.to_pickle('tmpdata/test_pre2.zip')

In [45]:
#train_pre3 = train_pre2.drop(["timestamp","wind_speed", "day", "hour"], axis=1)

In [50]:
#train_pre3.head(-1)

,building_id,meter,meter_reading,site_id,month
0,0,0,0.000,0,1.0
1,1,0,0.000,0,1.0
2,2,0,0.000,0,1.0
3,3,0,0.000,0,1.0
4,4,0,0.000,0,1.0
...,...,...,...,...,...
20216094,1443,0,64.950,15,12.0
20216095,1444,0,8.750,15,12.0
20216096,1445,0,4.825,15,12.0
20216097,1446,0,0.000,15,12.0


In [114]:
primary_use_int.head()

,timestamp,building_id,meter,primary_use,primary_use_int
0,2017-01-01,0,0,1,1
1,2017-01-01,1,0,1,1
2,2017-01-01,2,0,1,1
3,2017-01-01,3,0,1,1
4,2017-01-01,4,0,1,1


In [47]:
#train_pre3.to_pickle('tmpdata/train_pre3.zip')

In [115]:
 train_pre4 = pd.merge(train_pre2, primary_use_int[["timestamp", "building_id", "primary_use_int"]], how="left", on=["building_id", "timestamp"])
 gc.collect()

0

In [116]:
train_pre4.head()

,row_id,building_id,meter,timestamp,site_id,wind_speed,month,day,hour,primary_use_int
0,0,0,0,2017-01-01,0,3.6,1.0,1.0,0.0,1
1,1,1,0,2017-01-01,0,3.6,1.0,1.0,0.0,1
2,2,2,0,2017-01-01,0,3.6,1.0,1.0,0.0,1
3,3,3,0,2017-01-01,0,3.6,1.0,1.0,0.0,1
4,4,4,0,2017-01-01,0,3.6,1.0,1.0,0.0,1


In [117]:
train_pre4.to_pickle('tmpdata/test_pre4.zip')

tmp.head()

In [ ]:
# train_pre2.head()

In [ ]:
# train_pre2.corr()

In [4]:
train_pre4 = pd.read_pickle('tmpdata/train_pre4.zip')
train_pre5 = train_pre4.drop(["meter_reading", "timestamp"], axis=1)

x_train = pd.merge(train_pre5, building_metadata[["site_id", "building_id", "floor_count", "year_built"]],
                  how="left", on=["site_id", "building_id"])


y_train = train_pre4["meter_reading"]
gc.collect()

0

In [6]:
test_pre4 = pd.read_pickle('tmpdata/test_pre4.zip')
test_pre5 = test_pre4.drop(["timestamp"], axis=1)

x_test = pd.merge(test_pre5, building_metadata[["site_id", "building_id", "floor_count", "year_built"]],
                  how="left", on=["site_id", "building_id"])

y_test = test_pre4.drop(["timestamp"], axis=1)
gc.collect()

28

In [20]:
x_train.head(10)

,building_id,meter,site_id,wind_speed,month,day,hour,primary_use_int,floor_count,year_built
15738394,240,3,2,7.2,5.0,21.0,3.0,2,NaN,1964.0
32504696,771,0,6,0.0,10.0,11.0,7.0,2,NaN,NaN
26733864,171,0,2,2.1,8.0,23.0,3.0,3,NaN,1968.0
35986372,765,1,6,3.1,11.0,9.0,23.0,4,NaN,NaN
6639412,959,2,9,3.6,3.0,1.0,8.0,1,NaN,NaN
19192410,235,1,2,0.0,6.0,19.0,12.0,1,NaN,1970.0
16100095,264,1,2,4.1,5.0,24.0,5.0,3,NaN,1973.0
15164248,1386,1,15,4.1,5.0,16.0,5.0,4,NaN,1975.0
26684823,1295,2,14,6.2,8.0,22.0,16.0,3,NaN,NaN
36544901,467,0,3,3.6,11.0,14.0,20.0,8,NaN,NaN


In [21]:
import lightgbm as lgb

#categorical_cols = ['building_id','Month', 'meter','Hour','primary_use','DayOfWeek','DayOfMonth','site_id']
#categorical_cols = ['building_id', 'month', 'meter', 'wind_speed', 'floor_count', 'year_built', 'site_id']
categorical_cols = ['building_id', 'month', 'meter', 'hour', 'primary_use_int', 'site_id']

from sklearn.model_selection import train_test_split,KFold

x_train,x_test,y_train,y_test = train_test_split(x_train,y_train,test_size=0.25,random_state=42)

lgb_train = lgb.Dataset(x_train, y_train,categorical_feature=categorical_cols)
lgb_test = lgb.Dataset(x_test, y_test,categorical_feature=categorical_cols)

gc.collect()

54

In [22]:
params = {'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'objective': 'regression',
          'max_depth': -1,
          'learning_rate': 0.15,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'rmse',
          "verbosity": -1,
          'reg_alpha': 0.5,
          'reg_lambda': 0.5,
          'random_state': 47
         }

reg = lgb.train(params, lgb_train, num_boost_round=3000, valid_sets=[lgb_train, lgb_test], 
                early_stopping_rounds=100, verbose_eval = 100)


Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 36927.8	valid_1's rmse: 39153.8
[200]	training's rmse: 30746.6	valid_1's rmse: 33714.3
[300]	training's rmse: 27182	valid_1's rmse: 30735
[400]	training's rmse: 24774	valid_1's rmse: 28848.3
[500]	training's rmse: 23369.7	valid_1's rmse: 27976.5
[600]	training's rmse: 22385.4	valid_1's rmse: 27336.5
[700]	training's rmse: 21303.9	valid_1's rmse: 26572.7
[800]	training's rmse: 20480.1	valid_1's rmse: 25997.1
[900]	training's rmse: 19564.3	valid_1's rmse: 25355.9
[1000]	training's rmse: 18596.7	valid_1's rmse: 24648.4
[1100]	training's rmse: 18261.9	valid_1's rmse: 24407.7
[1200]	training's rmse: 17894.5	valid_1's rmse: 24200.5
[1300]	training's rmse: 17749.9	valid_1's rmse: 24064.5
[1400]	training's rmse: 17298.1	valid_1's rmse: 23763.5
[1500]	training's rmse: 16823.4	valid_1's rmse: 23474.9
[1600]	training's rmse: 16554.5	valid_1's rmse: 23291.2
[1700]	training's rmse: 16180.1	valid_1's rmse: 23011.7
[

In [ ]:
# モデルを保存する
# ['building_id', 'month', 'meter', 'wind_speed', 'floor_count', 'year_built', 'site_id']
filename = 'lightgbm_model_1.sav'
pickle.dump(model, open(filename, 'wb'))

# 保存したモデルをロードする
#loaded_model = pickle.load(open(filename, 'rb'))